# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df


Unnamed: 0         City  Cloudiness Country        Date  Humidity    Lat  \
0             0       Albany          26      US  1594701613        85  42.60   
1             1       Terney          46      RU  1594701781        71  45.05   
2             2       Pedasí          95      PA  1594701782        83   7.53   
3             3     Wakkanai          20      JP  1594701782        68  45.41   
4             4      Talnakh          75      RU  1594701782        93  69.49   
..          ...          ...         ...     ...         ...       ...    ...   
591         591          Ják          94      HU  1594701913        81  47.14   
592         592       Laurel          75      US  1594701899        94  39.10   
593         593       Omboué          95      GA  1594701914        87  -1.57   
594         594         Zeya          41      RU  1594701914        23  53.75   
595         595  Kangaatsiaq          91      GL  1594701914        91  68.31   

        Lng  Max Temp  Wind Speed  
0    -73.97     72.00        6.73  
1    136.61     66.94        5.46  
2    -80.03     79.88        5.23  
3    141.67     71.60       12.75  
4     88.40     53.60       15.66  
..      ...       ...         ...  
591   16.58     55.00        1.70  
592  -76.85     77.00        4.72  
593    9.26     71.71        8.48  
594  127.27     97.63        5.50  
595  -53.46     40.46        8.90  

[596 rows x 10 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

gmaps.configure(api_key=g_key)


locations = weather_df[["Lat", "Lng"]]


humidity = weather_df["Humidity"]

In [4]:

figure = gmaps.figure(center=(45.0, -10.0), zoom_level=3)

# Create heatmap layer
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
figure.add_layer(heatmap_layer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrow_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

narrow_city_df

Unnamed: 0             City  Cloudiness Country        Date  Humidity  \
66           66             Jalu           0      LY  1594701796        47   
252         252      Grand-Santi           0      GF  1594701837        96   
287         287        Antequera           0      ES  1594701844        64   
331         331        Contamana           0      PE  1594701855        91   
390         390             Gaza           0      PS  1594701812        70   
402         402      Lesozavodsk           0      RU  1594701870        67   
531         531        Clearlake           0      US  1594701617        54   
559         559  Tiznit Province           0      MA  1594701906        60   
564         564          Totness           0      SR  1594701907        88   
570         570          Awjilah           0      LY  1594701909        44   

       Lat     Lng  Max Temp  Wind Speed  
66   29.03   21.55     74.91        8.81  
252   4.25  -54.38     70.56        1.48  
287  37.02   -4.56     75.00        9.17  
331  -7.33  -75.02     72.01        0.76  
390  31.50   34.47     75.99        6.58  
402  45.48  133.42     78.26        5.14  
531  38.96 -122.63     73.00        3.56  
559  29.58   -9.50     71.10        1.19  
564   5.88  -56.32     76.87        5.44  
570  29.11   21.29     74.95        7.67

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Add a "Hotel Name" column to the DataFrame.
hotel_df = narrow_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

#result
hotel_df

City Country    Lat     Lng Hotel Name
66              Jalu      LY  29.03   21.55           
252      Grand-Santi      GF   4.25  -54.38           
287        Antequera      ES  37.02   -4.56           
331        Contamana      PE  -7.33  -75.02           
390             Gaza      PS  31.50   34.47           
402      Lesozavodsk      RU  45.48  133.42           
531        Clearlake      US  38.96 -122.63           
559  Tiznit Province      MA  29.58   -9.50           
564          Totness      SR   5.88  -56.32           
570          Awjilah      LY  29.11   21.29

In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
  
    params["location"] = f"{lat},{lng}"

   
    print(f"Finding Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    
    except (KeyError, IndexError):
        print("Missing field/result, so skipped.")
        
 

Finding Results for Index 66: Jalu.
Closest hotel in Jalu is Jalu Hotel.
Finding Results for Index 252: Grand-Santi.
Closest hotel in Grand-Santi is Hotel La Villa Couacou.
Finding Results for Index 287: Antequera.
Closest hotel in Antequera is Convento de la Magdalena.
Finding Results for Index 331: Contamana.
Closest hotel in Contamana is Hotel Zavala´S.
Finding Results for Index 390: Gaza.
Closest hotel in Gaza is فندق جراند بالاس Grand Palace Hotel.
Finding Results for Index 402: Lesozavodsk.
Closest hotel in Lesozavodsk is Hotel Chelyuskina.
Finding Results for Index 531: Clearlake.
Closest hotel in Clearlake is Best Western El Grande Inn.
Finding Results for Index 559: Tiznit Province.
Missing field/result, so skipped.
Finding Results for Index 564: Totness.
Closest hotel in Totness is STAATS LOGEERGEBOUW TOTNESS.
Finding Results for Index 570: Awjilah.
Missing field/result, so skipped.


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
figure.add_layer(markers)

# Display Map
figure


Figure(layout=FigureLayout(height='420px'))